In [134]:
'''Current issues for all functions:
    Fair Price Error (Make it so that is bid or ask is 0, fair price is 0.01)
    Get float to round upto 2 decimal places
    Make the entire thing wayyy more efficient
'''
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import numpy as np
import yfinance as yf
import time
from os import system, name

s = yf.Ticker("MMYT")
opt = s.option_chain('2020-08-20')
calls, puts = opt.calls, opt.puts
print(int(calls['volume'].mean()))
calls['Fair Price'] = [i if i >= 0.01 else 0.01 for i in (calls['ask']+calls['bid'])/2]
puts['Fair Price']  = [i if i >= 0.01 else 0.01 for i in (puts['ask'] + puts['bid'])/2]

13


In [135]:
def getCalls():
    c = calls[calls['volume']>=int(calls['volume'].mean())]
    l = c[['strike', 'Fair Price']]
    j = c[['strike', 'lastPrice']]
    if c['ask'].mean() != 0:
        m = l.set_index('strike').to_dict()
        n = m['Fair Price'] 
        print("Using Current Market Price")
    else:
        m = j.set_index('strike').to_dict()
        n = m['lastPrice']
        print("Using Last Price")
    return n

def getPuts():
    p = puts[puts['volume']>=int(puts['volume'].mean())]
    l = p[['strike', 'Fair Price']]
    j = p[['strike', 'lastPrice']]
    if p['ask'].mean() != 0:
        m = l.set_index('strike').to_dict()
        n = m['Fair Price'] 
        print("Using Current Market Price")
    else:
        m = j.set_index('strike').to_dict()
        n = m['lastPrice']
        print("Using Last Price")
    return n

def appendBest(best_ratio, l, s, lp, sp, r, mrx, mrw):
    best_ratio['Long(Buy)']=l
    best_ratio['Short(Sell)']=s
    best_ratio['Risk/Reward Ratio']=r
    best_ratio['Short Premium'] =sp
    best_ratio['Long Premium'] = lp
    best_ratio['maxRisk'] = mrx
    best_ratio['maxReward'] = mrw

    
def rnR(type, sL, sS, pL, pS):
    if type == 'callC' or type == 'putC':
        MaxReward = pS - pL
        MaxRisk = (sL - sS - MaxReward)
        if type == 'putC':
            MaxRisk *= -1
    if type == 'callD' or type == 'putD':
        MaxReward = sL - sS - pL - pS
        MaxRisk = pL - pS
        if type == 'callD':
            MaxReward *= -1
    return MaxRisk, MaxReward



In [136]:
def Call_Credit_Spread():
    
    n = getCalls()
    
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

# best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}

    # First loop to check what to short (sell)   
    for short in n:
        
        # Second loop to check what we're buying (long)
        for long in n:
            
            # In a credit call spread the long Strike cannot be lower or equal to the short strike price
            if long <= short:
                continue
            else:
                
                # Declaring variables to work with
                strikeShort = short
                strikeLong = long
                
                premiumShort = n[short]
                premiumLong = n[long]
                
                # Figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                z = rnR('callC', strikeLong, strikeShort, premiumLong, premiumShort)
                MaxRisk = z[0]
                MaxReward = z[1]
                
                if MaxReward>0:
                    
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        appendBest(best_ratio, long, short, premiumLong, premiumShort, Ratio, MaxRisk, MaxReward)
                        
                        
    # returning the final best trade available as a call credit spread              
    return print("Best Call Credit Spread: ", best_ratio)
        



In [137]:
def Put_Credit_Spread():
    
    n = getPuts()
    
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

    # best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    
    # First loop to check what to long (buy) 
    for long in n:
        
        # Second loop to check what we're selling(short)
        for short in n:
            
            # In a put credit spread the strike price of what we're buying cannot be higher than strike price of what we're selling
            if long >= short:
                continue
                
            else:
                
                #Decalring various variables
                strikeShort = short
                strikeLong = long
                premiumShort = n[short]
                premiumLong = n[long]
                
                #figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                z = rnR('putC', strikeLong, strikeShort, premiumLong, premiumShort)
                MaxRisk = z[0]
                MaxReward = z[1]
                
                if MaxReward>0:
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        appendBest(best_ratio, long, short, premiumLong, premiumShort, Ratio, MaxRisk, MaxReward)


                        
                        
    return print("Best Put Credit Spread: ", best_ratio)

In [138]:
def Put_Debit_Spread():
    
    n = getPuts()

#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

    # best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    
    # First loop to check what we're buying
    for long in n:
        
        # Second loop for what we're selling
        for short in n:
            
            # We don't want buying strike to be smaller than selling strike (Put debit, buy higher strike and sell lower strike)
            if long <= short:
                continue
            else:
                
                #Declaring variables
                strikeShort = short
                strikeLong = long
                premiumShort = n[short]
                premiumLong = n[long]
                
                # figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                z = rnR('putD', strikeLong, strikeShort, premiumLong, premiumShort)
                MaxRisk = z[0]
                MaxReward = z[1]
                
                
                if MaxReward>0:
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        appendBest(best_ratio, long, short, premiumLong, premiumShort, Ratio, MaxRisk, MaxReward)


                        
    return print("Best Put Debit Spread: ", best_ratio)

In [139]:
#Put_Debit_Spread()

In [140]:
def Call_Debit_Spread():
    
    n = getCalls()

#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    
    
    # First loop to check what we're buying
    for long in n:
        
        # Second loop for what we're selling
        for short in n:
            
            # We don't want buying strike to be higher than selling strike (Call debit, sell higher strike and buy lower strike)
            if long >= short:
                continue
            else:
                
                #Declaring variables
                strikeShort = short
                strikeLong = long
                premiumShort = n[short]
                premiumLong = n[long]
                

                z = rnR('callD', strikeLong, strikeShort, premiumLong, premiumShort)
                MaxRisk = z[0]
                MaxReward = z[1]
                
                                   
                if MaxReward > 0:
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        appendBest(best_ratio, long, short, premiumLong, premiumShort, Ratio, MaxRisk, MaxReward)


                        
                        
    return print("Best Call Debit Spread: ", best_ratio)

In [141]:
def finalSpreads():
    print("\t\t\t~~~~~~~~~~~~~~~~~~~~~~~~Bullish~~~~~~~~~~~~~~~~~~~~~~~~~ \n")
    Call_Debit_Spread()
    print("\n")
    Put_Credit_Spread()
    print("\n")
    print("\t\t\t~~~~~~~~~~~~~~~~~~~~~~~~Bearish~~~~~~~~~~~~~~~~~~~~~~~~~ \n")
    Put_Debit_Spread()
    print("\n")
    Call_Credit_Spread()

In [142]:
finalSpreads()

			~~~~~~~~~~~~~~~~~~~~~~~~Bullish~~~~~~~~~~~~~~~~~~~~~~~~~ 

Using Current Market Price
Best Call Debit Spread:  {'Short(Sell)': 35.0, 'Long(Buy)': 20.0, 'Risk/Reward Ratio': 0.014634146341463414, 'Short Premium': 0.075, 'Long Premium': 0.3, 'maxRisk': 0.22499999999999998, 'maxReward': 15.375}


Using Current Market Price
Best Put Credit Spread:  {'Short(Sell)': 35.0, 'Long(Buy)': 25.0, 'Risk/Reward Ratio': 2.015228426395939, 'Short Premium': 19.1, 'Long Premium': 9.25, 'maxRisk': 19.85, 'maxReward': 9.850000000000001}


			~~~~~~~~~~~~~~~~~~~~~~~~Bearish~~~~~~~~~~~~~~~~~~~~~~~~~ 

Using Current Market Price
Best Put Debit Spread:  {'Short(Sell)': 5.0, 'Long(Buy)': 7.5, 'Risk/Reward Ratio': -0.06024096385542168, 'Short Premium': 0.275, 'Long Premium': 0.15000000000000002, 'maxRisk': -0.125, 'maxReward': 2.075}


Using Current Market Price
Best Call Credit Spread:  {'Short(Sell)': 15.0, 'Long(Buy)': 17.5, 'Risk/Reward Ratio': 1.0408163265306125, 'Short Premium': 2.05, 'Long Premium': 0